In [ ]:
import pandas as pd
from PIL import Image
import tqdm
import os
# 读取meta.csv文件
root = 'outputs/aug_samples/cub/dreambooth-lora-augmentation-Multi7-Strength0.08'
csv_file = 'meta.csv'
df = pd.read_csv(os.path.join(root,csv_file))

# 创建一个新的DataFrame来保存有效的行
valid_rows = []

# 遍历每一行
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    image_path = os.path.join(root,'data',row['Path'])
    try:
        # 尝试加载图片
        img = Image.open(image_path)
        img.close()  # 关闭图片文件
        valid_rows.append(row)  # 如果成功加载图片，则保留该行
    except Exception as e:
        # 如果加载图片时出错，删除该图片文件
        os.remove(image_path)
        print(f"Deleted {image_path} due to error: {str(e)}")

# 创建新的DataFrame，只包含有效的行
valid_df = pd.DataFrame(valid_rows)

# 保存有效的DataFrame到新的CSV文件
valid_df.to_csv(os.path.join(root,'valid_meta.csv'), index=False)


## 删除非0.5strength的图片

In [ ]:
import re
import os
from collections import defaultdict
from tqdm import tqdm
from PIL import Image
import pandas as pd
import shutil

output_path = '/data/zhicai/code/Diff-Mix/outputs/aug_samples/cub/dreambooth-lora-mixup-Multi7-db_ti35000-Strength0.5'
rootdir = os.path.join(output_path,'data')
# 正则表达式模式来匹配路径中的各个部分
pattern_level_1 = r"(.+)"
pattern_level_2 = r"(.+)-(\d+)-(.+).png"

# Generate meta.csv for indexing images
data_dict = defaultdict(list)
for dir in os.listdir(rootdir):
    if not os.path.isdir(os.path.join(rootdir, dir)):
        continue
    match_1 = re.match(pattern_level_1, dir)
    first_dir = match_1.group(1).replace("_", " ") 
    for file in os.listdir(os.path.join(rootdir, dir)):
        match_2 = re.match(pattern_level_2, file)
        second_dir = match_2.group(1).replace("_", " ")
        num = int(match_2.group(2))
        floating_num = float(match_2.group(3))
        if floating_num != 0.5:
            # delete files with strength not 0.5
            os.remove(os.path.join(rootdir, dir, file))
            continue
        data_dict['First Directory'].append(first_dir)
        data_dict['Second Directory'].append(second_dir)
        data_dict['Number'].append(num)
        data_dict['Strength'].append(floating_num)
        data_dict['Path'].append(os.path.join(dir, file))

df = pd.DataFrame(data_dict)

# Validate generated images
valid_rows = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_path = os.path.join(output_path,'data',row['Path'])
    try:
        # 尝试加载图片
        img = Image.open(image_path)
        img.close()  
        valid_rows.append(row)  # 如果成功加载图片，则保留该行
    except Exception as e:
        # 如果加载图片时出错，删除该图片文件
        os.remove(image_path)
        print(f"Deleted {image_path} due to error: {str(e)}")

valid_df = pd.DataFrame(valid_rows)
csv_path = os.path.join(output_path, 'meta.csv')
valid_df.to_csv(csv_path, index=False)

print("DataFrame:")
print(df)